In [1]:
from music21 import converter, instrument, note, chord
import numpy as np
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pickle
from collections import defaultdict

In [2]:
#WITHOUT INSTRUMENT2
def read_midi(file_path): 
    notes = [] 
    notes_to_parse = None 

    try:
        midi= converter.parse(file_path)
        s2 = instrument.partitionByInstrument(midi)
        for part in s2.parts:

            #select elements of only piano
            notes_to_parse = part.recurse() 
            #finding whether a particular element is note or a chord
            for element in notes_to_parse:
                #note
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                #chord
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))

        return np.array(notes)
    except: 
        return np.array([])


def unique(np_array, array_type, unique_set): 
    unique_set_reverse = {n:idx for (idx, n) in unique_set.items()}
    new_array =[] 
    if array_type == 'x':
        for group in np_array: 
            temp = [] 
            for n in group: 
                temp.append(unique_set_reverse[n])
            new_array.append(temp)
    else: 
        for n in np_array: 
            new_array.append(unique_set_reverse[n])
    new_array = np.array(new_array)
    return new_array

def remove_rare(note_array, note_count, min_count = 20): 
    note_array = [i for i in note_array if note_count[i] > min_count]
    return np.array(note_array)

def get_tts(base_dir, pick_name = None, sample_num= 1000, timestep = 10): 
    song_files = [f'{base_dir}/{i}' for i in os.listdir(base_dir)][:sample_num]
    songs_notes = [read_midi(i) for i in tqdm(song_files, desc = 'Reading Midi Files')]
    songs_notes = [i for i in songs_notes if len(i) != 0]
    all_notes = [i for note_ in songs_notes for i in note_]
    note_counter = defaultdict(int)
    for i in all_notes: 
        note_counter[i]+= 1
    #remove rare words
    songs_notes = [remove_rare(i, note_counter) for i in songs_notes]
    all_notes = [i for note_ in songs_notes for i in note_]
    note_counter = defaultdict(int)

    all_notes_dict = {idx: n for idx, n in enumerate(set(all_notes))}
    x = [] 
    y = [] 

    pbar = tqdm(songs_notes, desc = 'Creating Timeseries')
    for idx_song, song in enumerate(pbar):
        for idx in range(timestep,len(song)-(timestep), 1): 
            x.append(song[idx:idx+timestep])
            y.append(song[idx + (timestep)])

    x = np.array(x)
    y = np.array(y)  
    
    new_x = unique(x, unique_set = all_notes_dict, array_type = 'x')
    new_y = unique(y, unique_set = all_notes_dict, array_type ='y')
    
    
    x_train, x_test, y_train, y_test = train_test_split(new_x, new_y, train_size = .92, random_state = 10, stratify = new_y)
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1],1)
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1],1)
    
    if pick_name:
        pick_tup= (x_train, x_test, y_train, y_test, all_notes_dict)
        pickle.dump(pick_tup, open(f'../Pickles/{pick_name}', 'wb'), protocol = 4)
                    
    return x_train, x_test, y_train, y_test, all_notes_dict
# test = get_tts('../Classical', sample_num = 300, pick_name= 'Classical_All.p')
# assert False
x_train, x_test, y_train, y_test, all_notes_dict = get_tts('../Classical', sample_num = 2000, pick_name= 'Classical_All1.p')

Creating Timeseries: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1549/1549 [00:02<00:00, 524.64it/s]


In [ ]:
test

In [ ]:
assert False

In [ ]:
#WITHOUT INSTRUMENT
def read_midi(file_path): 
    notes = [] 
    notes_to_parse = None 

    try:
        midi= converter.parse(file_path)
        s2 = instrument.partitionByInstrument(midi)
        for part in s2.parts:

            #select elements of only piano
            if 'Piano' in str(part): 
                notes_to_parse = part.recurse() 
                #finding whether a particular element is note or a chord
                for element in notes_to_parse:
                    #note
                    if isinstance(element, note.Note):
                        notes.append(str(element.pitch))
                    #chord
                    elif isinstance(element, chord.Chord):
                        notes.append('.'.join(str(n) for n in element.normalOrder))

        return np.array(notes)
    except: 
        return np.array([])


def unique(np_array, array_type, unique_set): 
    unique_set_reverse = {n:idx for (idx, n) in unique_set.items()}
    new_array =[] 
    if array_type == 'x':
        for group in np_array: 
            temp = [] 
            for n in group: 
                temp.append(unique_set_reverse[n])
            new_array.append(temp)
    else: 
        for n in np_array: 
            new_array.append(unique_set_reverse[n])
    new_array = np.array(new_array)
    return new_array

def get_tts(base_dir, pick_name = None, sample_num= 1000, timestep = 10): 
    song_files = [f'{base_dir}/{i}' for i in os.listdir(base_dir)][:sample_num]
    songs_notes = [read_midi(i) for i in tqdm(song_files, desc = 'Reading Midi Files')]
    songs_notes = [i for i in songs_notes if len(i) != 0]
    all_notes = [i for note_ in songs_notes for i in note_]
    note_counter = defaultdict(int)
    for i in all_notes: 
        note_counter[i]+= 1
    
    int_to_note = dict((idx, n) for idx, n in enumerate(all_notes))
    all_notes_dict = {idx: n for idx, n in enumerate(set(all_notes))}
    x = [] 
    y = [] 

    pbar = tqdm(songs_notes, desc = 'Creating Timeseries')
    for idx_song, song in enumerate(pbar):
        for idx in range(timestep,len(song)-(timestep), 1): 
            x.append(song[idx:idx+timestep])
            y.append(song[idx + (timestep)])

    x = np.array(x)
    y = np.array(y)  
    
    new_x = unique(x, unique_set = all_notes_dict, array_type = 'x')
    new_y = unique(y, unique_set = all_notes_dict, array_type ='y')
    
    
    x_train, x_test, y_train, y_test = train_test_split(new_x, new_y, train_size = .90, random_state = 10)
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1],1)
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1],1)
    
    if pick_name:
        pick_tup= (x_train, x_test, y_train, y_test, all_notes_dict)
        pickle.dump(pick_tup, open(f'../Pickles/{pick_name}', 'wb'), protocol = 4)
                    
    return x_train, x_test, y_train, y_test, all_notes_dict

x_train, x_test, y_train, y_test, all_notes_dict = get_tts('../Classical', sample_num = 250, pick_name= 'Classical_200.p')

In [ ]:
print(len(all_notes_dict))
all_notes_dict

In [ ]:
assert False

In [ ]:
#WITH INSTRUMNET
def read_midi(file_path): 
    print(file_path)
    notes = [] 
    notes_to_parse = None 
    instruments = []
    midi= converter.parse(file_path)

    s2 = instrument.partitionByInstrument(midi)
    for part in s2.parts:
        str_part = str(part)
        if str_part == '<music21.stream.Part>': 
            continue 
        else:

            cutoff_len = len('<music21.stream.Part ') 
            instru = str_part[cutoff_len:]
            instru = instru[:instru.find('>')]
            notes_to_parse = part.recurse() 
            for element in notes_to_parse: 
                if isinstance(element, note.Note): 
                    notes.append(str(element.pitch))
                    instruments.append(instru)
                elif isinstance(element, chord.Chord): 
                    notes.append('.'.join(str(n) for n in element.normalOrder))      
                    instruments.append(instru)
    return np.array(list(zip(instruments, notes)))

            
                    


def unique(np_array, array_type, unique_set): 
    unique_set_reverse = {n:idx for (idx, n) in unique_set.items()}
    new_array =[] 
    if array_type == 'x':
        for group in np_array: 
            temp = [] 
            for n in group: 
                n = ' '.join(n)
                temp.append(unique_set_reverse[n])
            new_array.append(temp)
    else: 
        for n in np_array: 
            n = ' '.join(n)
            new_array.append(unique_set_reverse[n])
    new_array = np.array(new_array)
    return new_array


def get_tts(base_dir, pick_name = None, timestep = 10): 
    song_files = [f'{base_dir}/{i}' for i in os.listdir(base_dir)]
    songs_notes = [read_midi(i) for i in tqdm(song_files, desc = 'Reading Midi Files')]
    all_notes = [' '.join(i) for note_ in songs_notes for i in note_]
    note_counter = defaultdict(int)
    for i in all_notes: 
        note_counter[i]+= 1
        
    all_notes_dict = {idx: n for idx, n in enumerate(set(all_notes))}
    
    x = [] 
    y = [] 

    pbar = tqdm(songs_notes, desc = 'Creating Timeseries')
    for idx_song, song in enumerate(pbar):
        for idx in range(timestep,len(song)-(timestep), 1): 
            x.append(song[idx:idx+timestep])
            y.append(song[idx + (timestep)])

    x = np.array(x)
    y = np.array(y)  
    
    new_x = unique(x, unique_set = all_notes_dict, array_type = 'x')
    new_y = unique(y, unique_set = all_notes_dict, array_type ='y')
    
    
    x_train, x_test, y_train, y_test = train_test_split(new_x, new_y, train_size = .90, random_state = 10)
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1],1)
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1],1)
    
    if pick_name:
        pick_tup= (x_train, x_test, y_train, y_test, all_notes_dict)
        pickle.dump(pick_tup, open(f'../Pickles/{pick_name}.p', 'wb'), protocol = 4)
                    
    return x_train, x_test, y_train, y_test, all_notes_dict

x_train, x_test, y_train, y_test, notes_dict = get_tts('../Techno', pick_name = 'Techno')

In [ ]:
x_train